## mindspore.ops.mean(x, axis=None, keep_dims=False) -〉Tensor
返回输入所有元素的平均值。
- 输入：
    * x: mindspore的tensor, 数据类型为Number。
    * axis: int、tuple(int)、list(int)或mindspore的tensor。
    * keepdims: bool。

- 返回：mindspore的tensor。

1、参数比较：
| mindspore   | torch       | jax         |
| :----:      | :----:      | :----:      |
| input       | input       | a           |
| axis        | dim         | axis        |
| keepdims    | keepdim     | dtype       |
|             | dtype       | out         |
|             | out         | keepdims    |
|             |             | where       |

* jax提供了where参数：是否要参与计算平均值。
* torch和jax都可以通过dtype设置返回的数据类型以及out的出参方式。

2、返回值比较

In [9]:
import numpy as np
import mindspore as ms
import torch
import jax.numpy as jnp

input = np.array([[1, 3, 4, 2],
                  [5, 2, 6, 3],
                  [8, 1, 2, 9]])

y1 = ms.ops.mean(ms.tensor(input), 1)
y3 = jnp.mean(input, 1)
print ('mindspore output:\n',y1)
print('\n')
print ('jax output:\n',y3)

mindspore output:
 [2 4 5]


jax output:
 [2.5 4.  5. ]


当input为int时，ms返回int，jax返回float，torch不接受int型作为input。

In [10]:
input = np.array([[1., 3, 4, 2],
                  [5, 2, 6, 3],
                  [8, 1, 2, 9]])

y1 = ms.ops.mean(ms.tensor(input), 1)
y2 = torch.mean(torch.tensor(input), 1)
y3 = jnp.mean(input, 1)
print ('mindspore output:\n',y1)
print('\n')
print ('torch output:\n',y2)
print('\n')
print ('jax output:\n',y3)

mindspore output:
 [2.5 4.  5. ]


torch output:
 tensor([2.5000, 4.0000, 5.0000], dtype=torch.float64)


jax output:
 [2.5 4.  5. ]


ms与jax不返回类型。

使用where，来确定计算范围时：

In [15]:
where=np.array([[1, 0, 0, 0],
                [0, 1, 0, 1],
                [1, 1, 1, 0]], dtype=bool)

y3 = jnp.mean(input, 1, keepdims=True, where=where)
print ('jax output:\n',y3)

jax output:
 [[1.       ]
 [2.5      ]
 [3.6666667]]


3、报错信息比较

In [16]:
y1 = ms.ops.mean(input, 1)

TypeError: Failed calling ReduceMean with "ReduceMean(keep_dims=bool)(x=<class 'numpy.ndarray'>, axis=int)".
The valid calling should be: 
"ReduceMean(keep_dims=<bool>)(x=<Tensor>, axis=<int, list of int, Tensor, tuple of int>)".

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore/ccsrc/pipeline/pynative/pynative_utils.cc:1294 PrintTypeCastError


In [17]:
y2 = torch.mean(input, 1)

TypeError: mean() received an invalid combination of arguments - got (numpy.ndarray, int), but expected one of:
 * (Tensor input, *, torch.dtype dtype)
 * (Tensor input, tuple of ints dim, bool keepdim, *, torch.dtype dtype, Tensor out)
 * (Tensor input, tuple of names dim, bool keepdim, *, torch.dtype dtype, Tensor out)


In [18]:
input = [2, 4, 3, 1]
y3 = jnp.mean(input, 1)

TypeError: mean requires ndarray or scalar arguments, got <class 'list'> at position 0.

当输入类型不正确时，报错信息torch简洁明确。建议ms优化。